### Check data correctness

In [25]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32

today = date.today().strftime('%Y-%m-%d')
# today = '2024-03-22'

In [26]:
# connection
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate = '{today}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
shortage_ext['HP_PN'] = shortage_ext['HP_PN'].str.replace('\n', ',', regex=False)
shortage_ext['FV'] = shortage_ext['FV'].str.replace('\n', ',', regex=False)

# download PNFV from SQL
cursor.execute("SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV = PNFV.rename({'Descr':'FV'})

#### Check last FD date is empty

In [27]:
# create the last FD null row
lastFDnull = shortage_ext[shortage_ext['last FD date'].isnull()].copy()

if len(lastFDnull)>0:
    lastFDnull['Issue'] = 'Last FD is blank'
    lastFDnull['last FD date'] = 'Blank'
    print('Last FD is blank:', len(lastFDnull))
else:
    lastFDnull = pd.DataFrame()

#### Check P1~P3 sum <> total shortage

In [28]:
# P1 + P2 + P3 != Total
QtyMismatch = []
for i, row in shortage_ext.iterrows():
    P123 = row['P1'] + row['Net P2'] + row['Net P3']
    total = row['Total Shortage Qty'] 
    if P123 != total:
        QtyMismatch.append(row)
    else:
        pass

if len(QtyMismatch) > 0:
    df_QtyMismatch = pd.DataFrame(QtyMismatch)
    df_QtyMismatch['Issue'] = 'P1~P3 sum <> total shortage'
else:
    df_QtyMismatch = pd.DataFrame()

print(f'P1~P3 sum != total shortage: {len(QtyMismatch)}')

P1~P3 sum != total shortage: 0


#### Check BT > P1

In [29]:
# BT qty > P1
BT_P1 = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    P1 = row['P1'] 
    if BT > P1:
        BT_P1.append(row)
    else:
        pass

if len(BT_P1) > 0:
    df_BT_P1 = pd.DataFrame(BT_P1)
    df_BT_P1['Issue'] = 'BT shortage > P1'
else:
    df_BT_P1 = pd.DataFrame()

print('BT shortage > P1:', len(BT_P1))

BT shortage > P1: 0


#### Check BT > total shortage

In [30]:
# BT > total shortage
BT_total = []
for i, row in shortage_ext.iterrows():
    BT = row['BT shortage']
    total = row['Total Shortage Qty'] 
    if BT > total:
        BT_total.append(row)
    else:
        pass

if len(BT_total) > 0:
    df_BT_total = pd.DataFrame(BT_total)
    df_BT_total['Issue'] = 'BT shortage > Total shortage'
else:
    df_BT_total = pd.DataFrame()

print('BT > total:', len(BT_total))

BT > total: 0


#### Check working on upside > total shortage

In [31]:
# working on upside > total shortage
upside_total = []
for i, row in shortage_ext.iterrows():
    upside = row['Working on upside']
    total = row['Total Shortage Qty'] 
    if upside > total:
        upside_total.append(row)
    else:
        pass

if len(upside_total) > 0:
    df_upside_total = pd.DataFrame(upside_total)
    df_upside_total['Issue'] = 'Working on upside > Total shortage'
else:
    df_upside_total = pd.DataFrame()

print('upside > total:', len(upside_total))

upside > total: 0


#### Check PN not in PNFV table

In [32]:
# PN not in PNFV table
PN_list = PNFV.PN.tolist()


df_PN_notExist = []
for index, row in shortage_ext.iterrows():
    hp_pns = row['HP_PN'].split(',')
    if all(hp_pn.strip() in PN_list for hp_pn in hp_pns):
        pass
    else:
        row['Issue'] = 'HP_PNs do not exist in the PNFV table'
        df_PN_notExist.append(row)

if len(df_PN_notExist) > 0:
    df_PN_notExist = pd.DataFrame(df_PN_notExist)
else:
    df_PN_notExist = pd.DataFrame()

print('PN not in PNFV:', len(df_PN_notExist))

PN not in PNFV: 1


#### Check FV not in PNFV table

In [33]:
# FV not in PNFV table
FV_list = PNFV.Descr.tolist()

df_FV_notExist = []
for index, row in shortage_ext.iterrows():
    fvs = row['FV'].split(';')
    if all(fv.strip() in FV_list for fv in fvs):
        pass
    else:
        row['Issue'] = 'FVs do not exist in the PNFV table'
        df_FV_notExist.append(row)

if len(df_FV_notExist) > 0:
    df_FV_notExist = pd.DataFrame(df_FV_notExist)
else:
    df_FV_notExist = pd.DataFrame()

print('FV not in PNFV:', len(df_FV_notExist))

FV not in PNFV: 1


#### Concat all result

In [34]:
# data type management
try:
    error_all = pd.concat([lastFDnull, df_QtyMismatch, df_BT_P1, df_BT_total, df_upside_total, df_FV_notExist, df_PN_notExist], axis=0)
    error_all['P1'] = error_all['P1'].astype(int)
    error_all['Net P2'] = error_all['Net P2'].astype(int)
    error_all['Net P3'] = error_all['Net P3'].astype(int)
    error_all['BT shortage'] = error_all['BT shortage'].astype(int)
    error_all['Working on upside'] = error_all['Working on upside'].astype(int)
    error_all['Total Shortage Qty'] = error_all['Total Shortage Qty'].astype(int)
except:
    error_all = pd.DataFrame()
    print('No error found')

In [35]:
# concat same errors
if len(error_all) > 0:
    # fist group by all column except issue, then agg function will aggreate the words
    concatenated_issues = error_all.groupby(list(error_all.columns.difference(['Issue']))).agg({'Issue': ', '.join}).reset_index()
    error_all = concatenated_issues.drop_duplicates()
    error_all = error_all[['BuyerName','Issue','ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN',]]
else:
    pass

# External vs. Internal shortage

In [40]:
# connection
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] = '{today}'")
internal = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])

In [44]:
# check if there is empty value in Commodity
if internal.Commodity.isna().any():
    print(internal[internal.Commodity.isna()])
else:
    print('No empty value in Commodity')

No empty value in Commodity


In [82]:
item_num_int = internal.groupby('Commodity')['FV Des'].nunique().reset_index()
shortage_sum_int = internal.groupby('Commodity')['Total Shortage Quantity'].sum().reset_index()
df_int = pd.DataFrame(item_num_int).merge(shortage_sum_int, on='Commodity', how='left')
df_int.rename(columns={'FV Des': 'FV', 'Total Shortage Quantity': 'Total Shortage Qty'}, inplace=True)
df_int[['FV','Total Shortage Qty']] = df_int[['FV','Total Shortage Qty']].astype(int)

In [83]:
item_num_ext = shortage_ext[shortage_ext['Total Shortage Qty'] != 0].groupby('Commodity')['FV'].nunique()
shortage_sum_ext = shortage_ext[shortage_ext['Total Shortage Qty'] != 0].groupby('Commodity')['Total Shortage Qty'].sum()
df_ext = pd.DataFrame(pd.concat([item_num_ext, shortage_sum_ext], axis=1).reset_index())
df_ext[['FV','Total Shortage Qty']] = df_ext[['FV','Total Shortage Qty']].astype(int)

In [86]:
df_compare = df_int.merge(df_ext, on='Commodity', how='outer', suffixes=('_int', '_ext'))
df_compare['Remark'] = np.where((df_compare['FV_int'] != df_compare['FV_ext']) | (df_compare['Total Shortage Qty_int'] != df_compare['Total Shortage Qty_ext']), 'Mismatch', 'Match')
df_compare

,Commodity,FV_int,Total Shortage Qty_int,FV_ext,Total Shortage Qty_ext,Remark
0,AC ADAPTOR,1.0,1176.0,1,1176,Match
1,CHIPSET,2.0,366.0,2,366,Match
2,CPU,39.0,69127.0,64,153658,Mismatch
3,EC,1.0,12046.0,1,12046,Match
4,GPU,1.0,1252.0,8,19837,Mismatch
5,LCD,18.0,261246.0,18,261240,Mismatch
6,Memory,8.0,78267.0,8,78267,Match
7,POWERCORDS,20.0,103818.0,20,103818,Match
8,SSS,2.0,596.0,2,596,Match
9,TS,12.0,29346.0,12,29346,Match


In [90]:
df_compare_2 = df_compare[df_compare['Remark'] == 'Mismatch']
df_compare_2.drop(columns=['Remark'], inplace=True)
for i in df_compare_2.columns:
    if i == 'Commodity':
        pass
    else:
        df_compare_2[i] = df_compare_2[i].fillna(0)
        df_compare_2[i] = df_compare_2[i].astype(int)
df_compare_2

C:\Users\hsudu\AppData\Local\Temp\ipykernel_25752\1868922595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compare_2.drop(columns=['Remark'], inplace=True)
C:\Users\hsudu\AppData\Local\Temp\ipykernel_25752\1868922595.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compare_2[i] = df_compare_2[i].fillna(0)
C:\Users\hsudu\AppData\Local\Temp\ipykernel_25752\1868922595.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,Commodity,FV_int,Total Shortage Qty_int,FV_ext,Total Shortage Qty_ext
2,CPU,39,69127,64,153658
4,GPU,1,1252,8,19837
5,LCD,18,261246,18,261240
11,CONNECTOR,0,0,5,324879
12,CardReader,0,0,3,8748
13,HDD,0,0,1,20
14,PSU,0,0,6,11022
15,WEBCAM,0,0,2,56962


# Send email

In [96]:
if (len(error_all)) | (len(df_compare_2)) > 0:
    # sent to list
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = '<' + today + '> GPS B/S shortage dashboard alert '

    # Step 1: Highlight only the BuyerName values
    error_all['BuyerName'] = error_all['BuyerName'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')

    # Step 2: Convert the DataFrame to HTML, allowing HTML content within cells
    html_error_all = error_all.to_html(escape=False, index=False)

    # Step 3: Highlight only the Commodity values
    df_compare_2['Commodity'] = df_compare_2['Commodity'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')
    # Step 4: Convert the DataFrame to HTML, allowing HTML content within cells
    html_compare_2 = df_compare_2.to_html(escape=False, index=False)

    # message --> use the html content from the previous step 1&2 to 
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>
    Here are items that need your amendment in shortage dashboard.<br>
    Please complete it by next workday and contact project team if any question, thank you.<br>
    * Due to database refresh timing gap, items you have done action might still be on the list here. Please ignore if you have done action. Let project team know if your amendment is not completed for days.<br><br>
    {html_error_all}<br>Below information is <b><font color = 'blue'>misaligned/ missing in external & internal reports</b></font>, please help to check and make amendment.<br>{html_compare_2}{signature}"""

    # make sure email fully initialized, and only write after <body>
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()

else:
    print('No error found, no email sent')

C:\Users\hsudu\AppData\Local\Temp\ipykernel_25752\3594710053.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_compare_2['Commodity'] = df_compare_2['Commodity'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')
